# Notes from call with Chris

## Objectives
1. Document where the objects are on the seabed and what type (dimensions and density) (physics) using google search tracking down indexes from state governments and environemental groups 
    * artificial reefs
    * epidimeological layers
* synthesize this into a density mapping, provide hard bottom for habitat
* humans are hardening the sea floor
2. What is the fate of these objects?
    * are they going to be buried?
    * where will they be married?
    * Are they going to shift in a storm?

## Location of Study: East Coast
* East coast has more research regarding hurricane sandy
* Area from NY to Greys Reefe (offshore Georgia)

## Data
### DBSeaBed
* http://instaar.colorado.edu/~jenkinsc/dbseabed/kml/
* Gridded format - NetCDF
### Second Data Source
* Chris will send us this in google drive folder. 


First technical issue:
We have lots of information how are we going to draw it together?
Bring tables and store them in raw state in excel. 
10 columns of the data that he wants to purify from those sources


In [37]:
# This will grow and change.
import os
import earthpy as et
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
from fiona.drvsupport import supported_drivers


In [12]:
# Download the google earth KML data from dbSeabed
et.data.get_data(
    url=("http://instaar.colorado.edu/~jenkinsc"
         "/dbseabed/kml/dbS_global.kml"))

'/Users/robynmarowitz/earth-analytics/data/earthpy-downloads/dbS_global.kml'

In [13]:
# go to /earth-analytics/data directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

# Define the path where this data will be held
dbseabed_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'dbS_global.kml')

In [14]:
# Artifical reefs data download
# Figure out why this isnt working 
et.data.get_data(
    url=("https://drive.google.com/file/d/1dCGaw7svfcwCAUiiZ_8aa6Zr83Hx2h8G/view?usp=share_link"))

# reef_kmz_pth = os.path.join(
#    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
#    'Reef_Material.kmz')

# Convert KMZ to KML


'/Users/robynmarowitz/earth-analytics/data/earthpy-downloads/view?usp=share_link'

In [15]:
et.data.get_data(
    url=("https://wrecks.nauticalcharts.noaa.gov/"
         "downloads/ENC_Wrecks.kml"))

ENC_wrecks_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'ENC_wrecks.kml')

In [34]:
# Read in the KML and trim it to north and south carolina
def read_kml(kml_fp):
    supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file(kml_fp, driver='KML')
    return gdf

dbS_gdf = read_kml(dbseabed_kml_pth)
type(dbS_gdf)
dbS_gdf

,Name,Description,geometry


In [33]:
# ENC Wrecks
ENC_gdf = read_kml(ENC_wrecks_kml_pth)
ENC_gdf


,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (-79.03783 9.56900 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (-78.87901 9.55749 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (-78.94357 9.55448 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (-72.54199 18.23128 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (-72.53418 18.22828 0.00000)
...,...,...,...
11490,kml_11491,<center><table><tr><th colspan='2' align='cent...,POINT Z (-87.21964 18.70609 0.00000)
11491,kml_11492,<center><table><tr><th colspan='2' align='cent...,POINT Z (-86.65335 21.26916 0.00000)
11492,kml_11493,<center><table><tr><th colspan='2' align='cent...,POINT Z (-170.65833 14.37500 0.00000)
11493,kml_11494,<center><table><tr><th colspan='2' align='cent...,POINT Z (-117.39005 32.26052 0.00000)


In [28]:
# Turn geometry into Lattitude and longitude, create desired format og
# Lat, Lon, Size, Description
ENC_df = pd.DataFrame(columns=['lat', 'lon', 'size', 'description'])
ENC_df['lon'] = ENC_gdf.geometry.x
ENC_df['lat'] = ENC_gdf.geometry.y
ENC_df

,lat,lon,size,description
0,9.569000,-79.037834,NaN,NaN
1,9.557486,-78.879013,NaN,NaN
2,9.554478,-78.943573,NaN,NaN
3,18.231279,-72.541992,NaN,NaN
4,18.228279,-72.534180,NaN,NaN
...,...,...,...,...
11490,18.706093,-87.219643,NaN,NaN
11491,21.269161,-86.653351,NaN,NaN
11492,14.375000,-170.658333,NaN,NaN
11493,32.260523,-117.390053,NaN,NaN


In [36]:
# TODO: we need to create a bounding box for the northern coastline of 
# north carolina and the souther coastline of south carolina and decide 
# how much ocean to include

NC_northeast_land_point = (36.541466, -77.121369)
SC_southest_land_point = (32.087495, -81.037622)
NC_water_point = (36.545849, -71.760165)
SC_water_point = (32.368586, -71.511922)
bounding_coords = [(36.541466, -77.121369), 
                   (36.541466, -71.760165), 
                   (32.087495, -71.511922),
                   (31.036502, -78.317663)]

